In [ ]:
import requests
import re
cookie = None

s = requests.Session()
login_uri = 'https://id.tsinghua.edu.cn/do/off/ui/auth/login/post/167ed2c25d7f176c20c79e341e2ccdf0/0?/login.do'
values = {'i_user': "xcd19", 'i_pass': "", 'atOnce': 'true'}
info = s.post(login_uri, data=values).text
ticket = re.findall('ticket=(.+?)"', info)
successful = len(ticket) > 0
if successful:
    res = s.get(f"https://cloud.tsinghua.edu.cn/tsinghua-auth/callback/?ticket={ticket[0]}")
    cookie = s.cookies

In [ ]:
response = requests.get("https://cloud.tsinghua.edu.cn/api/v2.1/repos/?type=mine", cookies=cookie)
dic = response.json()
global_repos = dic["repos"]

In [ ]:
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()


async def async_get_pixabay_image_url():
    data_ls = []
    async with aiohttp.ClientSession(cookies=cookie) as session:
        for i, repo in enumerate(global_repos):
            async with session.get(f'https://cloud.tsinghua.edu.cn/api/v2.1/repos/{repo["repo_id"]}/') as response:
                if response.status == 200:
                    data = await response.json()
                    await asyncio.sleep(0)
                    print(i)
                    data_ls.append(data)
                else:
                    print(f"fail for {repo}")
    
    return data_ls



loop = asyncio.get_event_loop()
import time
st = time.time()
ret = loop.run_until_complete(async_get_pixabay_image_url())
ed = time.time()
print(ret)
print(ed - st)


st = time.time()
for i, repo in enumerate(global_repos):
    response = requests.get(f'https://cloud.tsinghua.edu.cn/api/v2.1/repos/{repo["repo_id"]}/', cookies=cookie)
    if response.status_code == 200:
        data = response.json()
        print(i)
    else:
        print(f"fail for {repo}")
ed = time.time()
print(ed - st)


In [ ]:
import requests,re
s = requests.Session()

In [ ]:
r = s.post("https://mails.tsinghua.edu.cn/coremail/index.jsp?cus=1", 
           data={
                "uid": "xcd19",
                "domain": "mails.tsinghua.edu.cn",
                "password": "xcd1017140921",
                "action:login": ""
           },)
print(r.cookies)
sid = re.findall(r'sid = "(\w+)";', r.text)[0]
print(sid)

In [ ]:
            
requests.utils.add_dict_to_cookiejar(s.cookies, {
    "Coremail.sid":sid
})

In [ ]:
email_url = "https://mails.tsinghua.edu.cn/coremail/XT3/mbox/viewMailHTML.jsp?mid={mid}&partId=0&isSearch=&priority=&supportSMIME=&striptTrs=true&mboxa=&sandbox=1"
mid = "2:1tbiAgQJDVEw+IojiQAAm7"
print(email_url.format(mid=mid))
r = s.get(email_url.format(mid=mid), headers={
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
})
print(r.status_code)
# print(r.text)


In [ ]:
mid = "1:1tbiAQILAWSvNiFFZQAAs1" # 论文交流mid
adl_mid = "2%3a1tbiAgMPAWSaFzpEEQABsI" # adl mid
download_url = "https://mails.tsinghua.edu.cn/coremail/XT3/mbox/allDownload.jsp?sid={sid}&mid={mid}"
r = s.post(download_url.format(sid=sid, mid=adl_mid))
print(r.text)

In [ ]:
with open("x.eml", "wb") as f:
    f.write(r.content)
    

In [ ]:
import zipfile
from pathlib import Path

f = zipfile.ZipFile("/Users/xxy/Github/icloud-crawl/email/2019-11-25 14:54:49+2:1tbiAgQFB1Ew9+MQagAAmL.zip",'r')
for name in f.namelist():
    extract_path = Path(f.extract(name, "/Users/xxy/Github/icloud-crawl"))
    extract_path.rename(name.encode("cp437").decode("gbk"))
    
f.close()

In [5]:
import asyncio
from pyppeteer import launch

async def main():
    browser = await launch()
    page = await browser.newPage()
    await page.goto('https://example.com')
    await page.screenshot({'path': 'example.png'})
    await browser.close()

asyncio.get_event_loop().run_until_complete(main())

RuntimeError: This event loop is already running